In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Extração de informações de textos com  Generative AI na Vertex AI 

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

A extração de informações é um processo que pode ser feito manualmente ou automaticamente. A extração manual de informações é o processo de ler o documento e copiar informações para um novo documento. A extração automática de informações é o processo de usar um software para extrair informações do documento.

A extração de informações de textos pode ser usada para uma variedade de propósitos. Um propósito comum é converter documentos em um formato legível por máquina. Isso pode ser útil para armazenar documentos em um banco de dados ou para processar documentos com software. Outra finalidade comum é extrair informações de documentos. Isso pode ser útil para localizar informações específicas em um documento ou para resumir o conteúdo de um documento.

Modelos de linguagem grandes (LLMs) são bons para extração de informações porque são treinados em conjuntos de dados massivos de texto e código, o que lhes permite aprender as relações entre palavras e frases. Eles também podem entender o contexto do texto e gerar texto, o que lhes permite extrair informações que não são explicitamente declaradas ou preencher as lacunas no texto que estão faltando. As respostas dos LLMs também podem ser melhoradas por meio de métodos como prompts de poucos disparos.

Saiba mais sobre os prompts de extração na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/extraction-prompts).

### Objetivo

Neste tutorial, você aprenderá como usar modelos generativos para extrair as informações de textos trabalhando com os seguintes exemplos:
- Extração de especificações técnicas do Google Pixel
- Solução de problemas de WiFi
- Responder a perguntas como um personagem
- Convertendo uma lista de ingredientes para o formato JSON
- Organizar os resultados de uma extração de texto

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Generative AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI e da Cloud Translate API

In [ ]:
!pip install google-cloud-aiplatform google-cloud-translate --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
from vertexai.preview.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

#### Criando a função *wrapper* para utilizar os modelos em Português

Até o momento desde treinamento, as API do Generative AI Studio suportam somente interações no idioma inglês. Para fazermos interações utilizando o idioma português, vamos utilizar a [Cloud Translation API](https://cloud.google.com/translate) para traduzir as nossas solicitações do português para o inglês e as respostas da API de inglês para português.

In [ ]:
from google.cloud import translate

project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id


def traduza(texto, idioma_destino):
    client = translate.TranslationServiceClient()

    response = client.translate_text(
        parent=parent,
        contents=[texto],
        target_language_code=idioma_destino,
        mime_type="text/plain"
    )

    return response.translations[0].translated_text

## Extração de informações de texto

### Extração das especificações técnicas do Google Pixel

Neste exemplo, você irá extrair as especificações técnicas de um telefone Pixel do texto no formato JSON usando a API PaLM.

In [ ]:
prompt = traduza("""
Extrae las especificaciones técnicas del siguiente texto en formato JSON.

Texto: Google Nest WiFi, velocidad de red de hasta 1200 Mbps, frecuencias de 2,4 GHz y 5 GHz, protocolo WP3
JSON: {
    "producto":"Google Nest Wi-Fi",
    "velocidad":"1200Mbps",
    "frecuencias": ["2,4 GHz", "5 GHz"],
    "protocolo":"WP3"
}

Texto: Google Pixel 7, red 5G, 8 GB de RAM, procesador Tensor G2, 128 GB de almacenamiento, coral
JSON:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text, "es")
)

### Solução de problemas de WiFi

Neste exemplo, você pede ao LLM para responder a uma pergunta sobre a solução de problemas de um roteador Google WiFi com base na descrição das diferentes luzes de status no roteador. O modelo só poderá responder com o texto que foi fornecido, o que ajuda a evitar que gere respostas potencialmente incorretas. Aqui está como você pode fazer isso usando a API PaLM.

In [ ]:
prompt = traduza("""
Responda la pregunta utilizando el siguiente texto. Responda únicamente con el texto proporcionado.
Pregunta: ¿Qué debo hacer para arreglar mi WiFi desconectado? La luz de mi enrutador WiFi de Google es amarilla y parpadea lentamente.

Texto:
Color: Sin luz
Qué significa: el enrutador no tiene energía o la luz se ha atenuado en la aplicación.
Que hacer:
Asegúrese de que el cable de alimentación esté correctamente conectado al enrutador y a un tomacorriente que funcione.
Si su dispositivo ya está configurado y la luz está apagada, verifique la configuración de brillo de la luz en la aplicación.
Si aún no hay luz, comuníquese con el servicio de atención al cliente de Google WiFi.

Color: blanco sólido, sin luz, blanco sólido
Qué significa: el dispositivo se está iniciando.
Que hacer:
Espere a que el dispositivo arranque. Esto toma alrededor de un minuto. Cuando termine, parpadeará lentamente en blanco, lo que le permitirá saber que está listo para la configuración.

Color: blanco pulsante lento
Qué significa: El dispositivo está listo para la configuración.
Que hacer:
Use la aplicación Google Home para configurar su enrutador.

Color: blanco sólido
Qué significa: El enrutador está en línea y todo está bien.
Que hacer:
Usted está en línea. ¡Aprovechar!

Color: amarillo pulsante lento
Qué significa: Hay un error de red.
Que hacer:
Asegúrese de que el cable Ethernet esté conectado al enrutador y al módem y que ambos dispositivos estén encendidos. Puede ser necesario desconectar y volver a conectar cada dispositivo.

Color: Amarillo intermitente rápido
Qué significa: está presionando el botón de reinicio y reiniciando de fábrica este dispositivo.
Que hacer:
Si continúa presionando el botón de reinicio, después de unos 12 segundos, la luz se volverá amarilla fija. Cuando sea amarillo fijo, suelte el botón de restablecimiento de fábrica.

Color: amarillo sólido
Qué significa: el enrutador se está restableciendo a la configuración de fábrica.
Que hacer:
Esto puede tardar hasta 10 minutos. Cuando termine, el dispositivo se reiniciará y comenzará a parpadear en blanco, lo que le permitirá saber que está listo para la configuración.
Imagen Luz roja fija Rojo fijo Algo anda mal. Fallo crítico. Restablecimiento de fábrica del enrutador. Si la luz permanece roja, comuníquese con el servicio de atención al cliente de Wi-Fi.
""", "en")
print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text, "es")
)

### Responder a perguntas como um personagem

Agora, você instrui o LLM a fingir ser Klara, uma pessoa. Você também contará ao modelo sobre os traços de personalidade de Klara. Em seguida, você pedirá ao modelo que responda a uma pergunta como Klara responderia.

In [ ]:
prompt = traduza("""
Eres Klara.
Klara es gestora de inversiones.
Klara solo responde si Klara está segura de que es correcto.
Klara responde a la pregunta del usuario según los resúmenes de la página a continuación.
Klara genera la identificación de referencia donde Klara encontró la respuesta para cada oración en la respuesta.
Klara también resume la parte donde se encuentra la información.

Resúmenes de página: ['ID de referencia 1. UBS quiere ser visto como un banco global con raíces suizas, no solo como un banco europeo. El nuevo presidente de la junta, \
                     Colm Kelleher y el director ejecutivo Ralph Hamers celebraron una serie de reuniones con influyentes gestores de fondos de EE. UU. para aumentar sus participaciones \
                     en el banco. UBS es uno de los bancos europeos más valiosos con una relación precio/valor contable de 1.', 'ID de referencia 2. Credit Suisse vende \
                     Participación del 30% en la gestora de activos suiza Energy Infrastructure Partners. EIP se especializa en inversiones de capital a largo plazo \
                     para activos de infraestructura renovable y de energía a gran escala. La transacción puede formar parte de un esfuerzo continuo por parte del banco suizo para \
                     reducir aún más la brecha de capital.' El príncipe Mohammed inyectaría alrededor de 500 millones de dólares en la escisión de CS First Boston. Otros inversores \
                     puede incluir al exjefe de Barclays, Bob Diamonds, Atlas Merchant Capital. El Banco Nacional Saudita ya posee una participación del 9,9 por ciento en \
                     institución suiza problemática. . The Wall Street Journal informó que Apollo también está interesado en invertir en CS First Boston. La división \
                     banco de inversión banco de inversión está a punto de convertirse en una nueva unidad .', 'ID de referencia 5. Banco privado \
                     Pictet, con sede en Ginebra, ha contratado a un nuevo equipo en su negocio de inversión en el mercado privado de miles de millones de dólares. Edmund Buckley, \
                     Nikolaus Hubmann, Sean Howard, Jan Dreesen y Hugo Hickson trabajarán en el espacio de inversión directa de capital privado en el futuro. O \
                     El compromiso del equipo de Buckley puede verse como un golpe para el socio de Pictet, Elif Aktuğ.', 'ID de referencia 6. UBS quiere ser visto\
                     como un banco global con raíces suizas, no solo como un banco europeo. Nuevo presidente Colm Kelleher y CEO Ralph Hamers \
                     celebró una serie de reuniones con influyentes gestores de fondos estadounidenses para aumentar sus participaciones en el banco. UBS es uno de los bancos \
                     europeos más valiosos con una relación precio-valor contable de 1 .', 'ID de referencia 7. Apollo Global Management, con sede en EE. UU., es \
                     entre un grupo de compañías financieras en conversaciones con Credit Suisse sobre una participación en el renovado banco de inversión. La pared
                     Street Journal informó que Apollo también está interesado en invertir en CS First Boston. La división de banca de inversión de \
                     la inversión se dividirá en una nueva unidad.', 'ID de referencia 8. Filial de Apex con sede en Luxemburgo Europa \
                     Depositary Bank (EDB) y el especialista en titulización Gentwo se asocian. EDB y Gentwo ofrecerán agentes y servicios de pago \
                     servicios bancarios a terceros inversores en todo el mundo. Gentwo desarrolla plataformas para gestores de activos, bancos, family office y \
                     capitalistas de riesgo.', 'Referencia ID 9. El SNB ha seguido planes para adquirir participaciones en instituciones financieras europeas y \
                     americanos durante algún tiempo. Credit Suisse, Julius Baer, ​​Standard Chartered y Asian DBS Group también fueron mencionados como posibles objetivos.', \
                     'Referencia ID 10. Zuercher Kantonalbank tiene la intención de expandir su negocio de banca privada, incluso en el extranjero. El CEO Urs Baumann ve a Credit\
                     Suisse como socio confiable para el banco estatal. Baumann seguirá centrándose en reducir la dependencia de los bancos del negocio de \
                     diferencial de interés en un intento de diversificar sus negocios.']

Pregunta del usuario: ¿Los bancos globales están invirtiendo en iniciativas de sostenibilidad?
Respuesta de Klara:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=40, top_p=0.8).text, "es")
)

### Convertendo uma lista de ingredientes para o formato JSON

Suponha que você queira listar ingredientes em receitas para inserir em um banco de dados, o que requer uma saída bem formatada como JSON. Isso pode ser feito usando um LLM da seguinte maneira:

In [ ]:
prompt = traduza("""
Extraiga los ingredientes de la siguiente receta. Devolver ingredientes en formato JSON con claves: ingrediente, cantidad, tipo.

Ingredientes:
* 1 cucharada de aceite de oliva
* 1 cebolla picada
* 2 zanahorias picadas
* 2 tallos de apio picados
* 1 cucharadita de comino molido
* 1/2 cucharadita de cilantro molido
* 1/4 cucharadita de cúrcuma en polvo
* 1/4 cucharadita de pimienta de cayena (opcional)
* Sal y pimienta para probar
* 425g de frijoles negros, lavados y escurridos
* 425g de frijoles, lavados y escurridos
* 420 g de tomates cortados en cubos de lata, sin escurrir
* Lata de 280 g de tomates picados con pimientos verdes, sin escurrir
* 4 tazas de caldo de verduras
* 1 taza de cilantro fresco picado
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text, "es")
)

### Organizando os resultados de uma extração de informações

Nesta seção, você extrai as informações desejadas de um bloco de texto e as organiza de maneira estruturada, como separar por vírgulas. Aqui você usa prompt few-shot para orientar o modelo a formatar suas saídas para serem separadas por vírgulas.

In [ ]:
prompt = traduza("""
Mensaje: Rachel Green (Jennifer Aniston), una mujer protegida pero amistosa, huye del día de su boda y de una vida rica pero insatisfactoria, y conoce a su amiga de la infancia Monica Geller (Courteney Cox), una chef tensa pero cariñosa.
Rachel se convierte en camarera en el café Central Perk en West Village después de mudarse al departamento de Monica arriba de Central Perk y unirse al grupo de amigas solteras de Monica en sus 20:
la excompañera de cuarto Phoebe Buffay (Lisa Kudrow), una incómoda masajista y música; el vecino Joey Tribbiani (Matt LeBlanc), un actor testarudo pero leal y trabajador; El compañero de cuarto de Joey, Chandler Bing (Matthew Perry),
un procesador de datos sarcástico y autocrítico; y el hermano mayor de Monica y compañero de cuarto de la universidad de Chandler, Ross Geller (David Schwimmer), un paleontólogo dulce pero inseguro.

Extraiga los personajes y los actores que los interpretaron del mensaje anterior:
Rachel Green - Jennifer Aniston, Monica Geller - Courteney Cox, Phoebe Buffay - Lisa Kudrow, Joey Tribbiani - Matt LeBlanc, Chandler Bing - Matthew Perry, Ross Geller - David Schwimmer

Mensaje: Juegos como el ajedrez, el póquer, el Go y muchos videojuegos siempre han sido un terreno fértil para la investigación de la IA. Diplomacy es un juego de comercio y creación de alianzas para siete jugadores que se juega en un antiguo mapa de la Europa dividida.
en provincias, donde cada jugador controla varias unidades (reglas de Diplomacia). En la versión estándar del juego, llamada Press Diplomacy, cada turno incluye una fase de negociación, después de la cual todos los jugadores revelan sus
movimientos seleccionados simultáneamente. El corazón de Diplomacia es la fase de negociación, donde los jugadores intentan llegar a un acuerdo sobre sus próximos movimientos. Por ejemplo, una unidad puede apoyar a otra unidad, lo que le permite superar la resistencia de otras unidades,
como se ilustra aquí: Los enfoques computacionales de la diplomacia se han investigado desde la década de 1980, muchos de los cuales se exploraron en una versión más simple del juego llamada Diplomacia sin prensa, donde la comunicación estratégica entre
los jugadores no están permitidos. Los investigadores también propusieron protocolos comerciales amigables con la computadora, a veces llamados \342\200\234Restricted-Press\342\200\235.

Extrae la definición de Diplomacia:
Un juego de comercio y construcción de alianzas para siete jugadores.


Mensaje: En 2016, cuando no usábamos simulación y usábamos una pequeña configuración de laboratorio de robot industrial para aprender a recoger objetos pequeños como juguetes, llaves y artículos domésticos cotidianos, tomó el equivalente de
cuatro meses para que un robot aprenda un agarre simple con una tasa de éxito del 75%. Hoy, un solo robot aprende a realizar una tarea compleja como abrir puertas con una tasa de éxito del 90 % en menos de un día.
del aprendizaje del mundo real. Aún más emocionante, mostramos que podemos tomar los algoritmos y los aprendizajes de abrir puertas y aplicarlos a una nueva tarea: enderezar las sillas en nuestros cafés. Este progreso nos da
esperamos que nuestro intento de construir robots de aprendizaje de propósito general sea posible.

Extraiga las tasas de éxito de los robots en 2016 y hoy, respectivamente:
75%, 90%

Mensaje: CapitalG se fundó hace una década para empoderar a los emprendedores con Alphabet y la experiencia inigualable de crecimiento de Google.
Tenemos el privilegio de compartir las lecciones aprendidas al ayudar a escalar Google, Stripe, Airbnb, CrowdStrike, Databricks y Zscaler con la próxima ola de empresas tecnológicas generacionales, quizás incluida la suya.
Alphabet es nuestro único LP y proporciona capital paciente a largo plazo. Como fondo de crecimiento independiente, nuestras prioridades están alineadas con nuestros emprendedores. Las empresas de CapitalG han logrado adaptarse al mercado de productos y están listas para escalar. Mantenemos una cartera pequeña y enfocada para que cada empresa reciba un capital sustancial y apoyo práctico.

Extracto de las empresas financiadas por CapitalG:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text, "es")
)

Como você pode ver na saída acima, com base no prompt few-shot, o LLM gerou a lista de nomes das empresas financiadas pelo CapitalG no formato separado por vírgulas.